In [1]:
import EEGExtract as eeg
import numpy as np
from share import datasets_basic_infos, ROOT_VOTING_SYSTEM_PATH
from data_loaders import load_data_labels_based_on_dataset


In [2]:
dataset_name = "aguilera_traditional"

In [3]:
version_name = "" # To keep track what the output processing alteration went through

# Folders and paths
subject_id = 1
dataset_foldername = dataset_name + "_dataset"
computer_root_path = ROOT_VOTING_SYSTEM_PATH + "/Datasets/"
data_path = computer_root_path + dataset_foldername
dataset_info: dict = datasets_basic_infos[dataset_name]

_, eegData, labels = load_data_labels_based_on_dataset(dataset_info, subject_id, data_path)
# AFTER: 3D np array [chans x ms x epochs] 
# BEFORE: eegData: 3D np array [epochs x chans x ms] 
eegData = np.transpose(eegData, (0, 2, 1))
eegData.shape

Used Annotations descriptions: ['OVTK_StimulationId_Label_01', 'OVTK_StimulationId_Label_02', 'OVTK_StimulationId_Label_03', 'OVTK_StimulationId_Label_04', 'OVTK_StimulationId_Label_05']
Not setting metadata
120 matching events found
Setting baseline interval to [0.0, 1.4] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 120 events and 701 original time points ...
0 bad epochs dropped
Not setting metadata
120 matching events found
No baseline correction applied
0 projection items activated


C:\Users\rosit\Documents\workprojects\bci_complete\voting_system_platform\processing_eeg_methods\data_loaders.py:28: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  raw.set_montage(mne.channels.read_custom_montage(channel_location))


(120, 701, 24)

In [4]:
fs = dataset_info["sample_rate"]

In [5]:
feature_list = []

## Complexity Features

In [6]:
#Shannon Entropy
ShannonRes = eeg.shannonEntropy(eegData, bin_min=-200, bin_max=200, binWidth=2)

In [7]:
#Tsalis Entropy (n=10)
# tsalisRes = eeg.tsalisEntropy(eegData, bin_min=-200, bin_max=200, binWidth=2, orders=list(range(1,10+1)))

In [8]:
# Subband Information Quantity
# delta (0.5–4 Hz)
eegData_delta = eeg.filt_data(eegData, 0.5, 4, fs)
ShannonRes_delta = eeg.shannonEntropy(eegData_delta, bin_min=-200, bin_max=200, binWidth=2)
# theta (4–8 Hz)
eegData_theta = eeg.filt_data(eegData, 4, 8, fs)
ShannonRes_theta = eeg.shannonEntropy(eegData_theta, bin_min=-200, bin_max=200, binWidth=2)
# alpha (8–12 Hz)
eegData_alpha = eeg.filt_data(eegData, 8, 12, fs)
ShannonRes_alpha = eeg.shannonEntropy(eegData_alpha, bin_min=-200, bin_max=200, binWidth=2)
# beta (12–30 Hz)
eegData_beta = eeg.filt_data(eegData, 12, 30, fs)
ShannonRes_beta = eeg.shannonEntropy(eegData_beta, bin_min=-200, bin_max=200, binWidth=2)
# gamma (30–100 Hz)
eegData_gamma = eeg.filt_data(eegData, 30, 100, fs)
#ShannonRes_gamma = eeg.shannonEntropy(eegData_gamma, bin_min=-200, bin_max=200, binWidth=2)

In [9]:
# Cepstrum Coefficients (n=2)
# CepstrumRes = eeg.mfcc(eegData=eegData, fs=fs,order=2)

In [10]:
# Lyapunov Exponent
# LyapunovRes = eeg.lyapunov(eegData)

In [11]:
# Fractal Embedding Dimension
# HiguchiFD_Res  = eeg.hFD(eegData[0,:,0],3)

In [12]:
# Hjorth Mobility
# Hjorth Complexity
# HjorthMob, HjorthComp = eeg.hjorthParameters(eegData)

In [13]:
# False Nearest Neighbor
# FalseNnRes = eeg.falseNearestNeighbor(eegData)

In [14]:
# ARMA Coefficients (n=2)
# armaRes = eeg.arma(eegData,order=2)

## Category Features

In [15]:
# Median Frequency
medianFreqRes = eeg.medianFreq(eegData,fs)

In [16]:
# δ band Power
bandPwr_delta = eeg.bandPower(eegData, 0.5, 4, fs)
# θ band Power
bandPwr_theta = eeg.bandPower(eegData, 4, 8, fs)
# α band Power
bandPwr_alpha = eeg.bandPower(eegData, 8, 12, fs)
# β band Power
bandPwr_beta = eeg.bandPower(eegData, 12, 30, fs)
# γ band Power
#bandPwr_gamma = eeg.bandPower(eegData, 30, 100, fs)

In [17]:
# Standard Deviation
std_res = eeg.eegStd(eegData)

In [18]:
# α/δ Ratio
ratio_res = eeg.eegRatio(eegData,fs)

In [19]:
# Regularity (burst-suppression)
regularity_res = eeg.eegRegularity(eegData,fs)

In [20]:
# Voltage < 5μ
volt05_res = eeg.eegVoltage(eegData,voltage=5)
# Voltage < 10μ
volt10_res = eeg.eegVoltage(eegData,voltage=10)
# Voltage < 20μ
volt20_res = eeg.eegVoltage(eegData,voltage=20)

In [21]:
# Diffuse Slowing
df_res = eeg.diffuseSlowing(eegData)

In [22]:
# Spikes
minNumSamples = int(70*fs/1000)
spikeNum_res = eeg.spikeNum(eegData,minNumSamples)

In [23]:
# Delta burst after Spike
deltaBurst_res = eeg.burstAfterSpike(eegData,eegData_delta,minNumSamples=7,stdAway = 3)

In [24]:
# Sharp spike
sharpSpike_res = eeg.shortSpikeNum(eegData,minNumSamples)

In [25]:
# Number of Bursts
numBursts_res = eeg.numBursts(eegData,fs)

In [26]:
# Burst length μ and σ
burstLenMean_res,burstLenStd_res = eeg.burstLengthStats(eegData,fs)

C:\Users\rosit\Documents\workprojects\bci_complete\venv_bci\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\rosit\Documents\workprojects\bci_complete\venv_bci\lib\site-packages\numpy\core\_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [27]:
# Burst Band Power for δ
burstBandPwrAlpha = eeg.burstBandPowers(eegData, 0.5, 4, fs)

In [28]:
# Number of Suppressions
numSupps_res = eeg.numSuppressions(eegData,fs)

In [29]:
# Suppression length μ and σ
suppLenMean_res,suppLenStd_res = eeg.suppressionLengthStats(eegData,fs)

##    Connectivity features

In [30]:
# Coherence - δ
coherence_res = eeg.coherence(eegData,fs)

In [31]:
#import importlib
#importlib.reload(eeg)

In [32]:
feature_list = []
feature_list.append(ShannonRes)
feature_list.append(ShannonRes_delta)
feature_list.append(ShannonRes_theta)
feature_list.append(ShannonRes_alpha)
feature_list.append(ShannonRes_beta)
#feature_list.append(ShannonRes_gamma)

feature_list.append(bandPwr_delta)
feature_list.append(bandPwr_theta)
feature_list.append(bandPwr_alpha)
feature_list.append(bandPwr_beta)
#feature_list.append(bandPwr_gamma)

feature_list.append(std_res)
feature_list.append(ratio_res)
feature_list.append(regularity_res)
feature_list.append(volt05_res)
feature_list.append(volt10_res)
feature_list.append(volt20_res)
feature_list.append(df_res) # all 0
feature_list.append(spikeNum_res) # all 0
feature_list.append(deltaBurst_res)
feature_list.append(sharpSpike_res)
feature_list.append(numBursts_res)
feature_list.append(burstLenMean_res)
feature_list.append(burstLenStd_res)
feature_list.append(numSupps_res)
feature_list.append(coherence_res)

In [33]:
feature_arr  = np.array(feature_list)
feature_arr = np.transpose(feature_arr, (1, 0, 2))
feature_arr.shape

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (24,) + inhomogeneous part.

In [ ]:
len(feature_list)

In [ ]:
# # https://pyod.readthedocs.io/en/latest/pyod.models.html
# from pyod import models
# from pyod.models import hbos,auto_encoder,lof,so_gaal,lscp,vae,abod,ocsvm,xgbod,pca
# clf = hbos.HBOS(n_bins=17, alpha=0.07, tol=0.5,contamination=.15)
# clf.fit(feature_arr)

In [ ]:
from data_utils import get_best_classificator_and_test_accuracy, ClfSwitcher
from sklearn.pipeline import Pipeline
from pyriemann.estimation import ERPCovariances, XdawnCovariances, Covariances
from pyriemann.tangentspace import TangentSpace

classifier_list = []
accuracy_list = []
for feature_array in feature_list:
    individual_classifier, individual_acc = get_best_classificator_and_test_accuracy(feature_array, labels, Pipeline([('clf', ClfSwitcher())]))

In [ ]:
# cohen_kappa_score(artifacts, clf.labels_) # Interesting